# Optimizing Parameters & Save, Load The Model

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import torchvision.models as models

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


In [2]:
"""
Hyperparameters
 - epoch: 데이터셋 반복 횟수
 - batch size: 파라미터 갱신 전 데이터 샘플 수
 - Learning Rate = 학습 속도
"""
epochs = 5
batch_size = 64
learning_rate = 1e-3

In [4]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [6]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308625  [    0/60000]
loss: 2.290267  [ 6400/60000]
loss: 2.280760  [12800/60000]
loss: 2.272119  [19200/60000]
loss: 2.249013  [25600/60000]
loss: 2.234174  [32000/60000]
loss: 2.239443  [38400/60000]
loss: 2.210549  [44800/60000]
loss: 2.217680  [51200/60000]
loss: 2.183870  [57600/60000]
Test Error: 
 Accuracy: 41.5%, Avg loss: 2.172293 

Epoch 2
-------------------------------
loss: 2.188783  [    0/60000]
loss: 2.169204  [ 6400/60000]
loss: 2.123444  [12800/60000]
loss: 2.134387  [19200/60000]
loss: 2.077862  [25600/60000]
loss: 2.036006  [32000/60000]
loss: 2.059336  [38400/60000]
loss: 1.983547  [44800/60000]
loss: 2.006010  [51200/60000]
loss: 1.931202  [57600/60000]
Test Error: 
 Accuracy: 48.4%, Avg loss: 1.916660 

Epoch 3
-------------------------------
loss: 1.955519  [    0/60000]
loss: 1.914598  [ 6400/60000]
loss: 1.810316  [12800/60000]
loss: 1.844800  [19200/60000]
loss: 1.731309  [25600/60000]
loss: 1.692370  [32000/600

In [12]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')


In [14]:
model = models.vgg16()
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [15]:
# 모델의 형태도 같이 save, load하려면
torch.save(model, 'model.pth')
model = torch.load('model.pth')